In [1]:
import pandas as pd
from pvlib import solarposition
from pvlib import irradiance

# Issues with DIRINT related to Datetime 

In [38]:
df = pd.read_csv('debug_era5_input.csv')  # Debug line to inspect input data
df['valid_time'] = pd.to_datetime(df['valid_time']) 

latitude = 47.4
longitude = 8.51

In [39]:
df

,Unnamed: 0,valid_time,u10,v10,d2m,t2m,ssrd,strd,skt,sp,tp
0,0,1994-01-01 00:00:00,-0.122040,1.759476,275.80963,276.88788,0.0,1186992.0,275.91016,94036.030,2.336785e-03
1,1,1994-01-01 01:00:00,-0.596008,1.568008,275.52438,276.56714,0.0,1164212.2,275.65326,93926.234,1.289132e-03
2,2,1994-01-01 02:00:00,-0.467590,1.085648,275.34610,276.51862,0.0,1167222.8,275.54370,93853.780,1.078871e-03
3,3,1994-01-01 03:00:00,1.001633,0.477585,275.10352,276.23584,0.0,1143113.0,275.06958,93886.086,1.883335e-03
4,4,1994-01-01 04:00:00,2.247452,0.525696,275.02140,275.09820,0.0,1162854.0,274.91820,94028.640,2.710547e-03
...,...,...,...,...,...,...,...,...,...,...,...
271747,271747,2024-12-31 19:00:00,0.669800,0.732730,269.74738,270.57947,0.0,838775.0,266.34216,96311.390,5.960465e-07
271748,271748,2024-12-31 20:00:00,0.614334,1.052210,269.50607,270.82712,0.0,834820.0,265.80786,96325.430,0.000000e+00
271749,271749,2024-12-31 21:00:00,0.601471,1.140213,269.43440,270.95270,0.0,851893.0,265.82153,96270.280,0.000000e+00
271750,271750,2024-12-31 22:00:00,0.638824,0.786973,269.85510,271.68286,0.0,843352.0,265.76416,96232.190,0.000000e+00


In [13]:
# Extract time components
df['Year'] = df['valid_time'].dt.year
df['Month'] = df['valid_time'].dt.month
df['Day'] = df['valid_time'].dt.day
df['Hour'] = df['valid_time'].dt.hour
df['Minute'] = 0  # ERA5 is hourly

# Remove leap days if specified
df = df[~((df['Month'] == 2) & (df['Day'] == 29))].reset_index(drop=True)

# Initialize output columns
output = pd.DataFrame()
output['Year'] = df['Year']
output['Month'] = df['Month']
output['Day'] = df['Day']
output['Hour'] = df['Hour']
output['Minute'] = df['Minute']

# Add location information if provided
if latitude is not None:
    output['Latitude'] = latitude
if longitude is not None:
    output['Longitude'] = longitude

# Temperature (K to °C)
if 't2m' in df.columns:
    output['Temperature'] = df['t2m'] - 273.15

if 'd2m' in df.columns:
    output['Dew Point'] = df['d2m'] - 273.15

# Pressure (Pa to hPa)
if 'sp' in df.columns:
    output['Pressure'] = df['sp'] / 100.0

# Solar radiation (J/m² to W/m² - divide by 3600 for hourly accumulation)
if 'ssrd' in df.columns:
    # ERA5 provides accumulated radiation, need hourly average
    output['GHI'] = df['ssrd'] / 3600.0


In [16]:

# Reconstruct datetime index from output DataFrame (after leap day removal)
times = pd.DatetimeIndex(pd.to_datetime(output[['Year', 'Month', 'Day', 'Hour', 'Minute']]))

# Calculate solar position
solpos = solarposition.get_solarposition(times, latitude, longitude)

# Calculate DNI using DIRINT method
# Pressure needs to be in Pa (convert from hPa)
# Use day of year instead of datetime to avoid pandas version issues
doy = times.dayofyear

# Reset index to ensure proper Series alignment
ghi_series = pd.Series(output['GHI'].values, index=times)
pressure_series = pd.Series(output['Pressure'].values * 100, index=times)
temp_dew_series = pd.Series(output['Dew Point'].values, index=times)

dni_dirint = irradiance.dirint(
    ghi_series, 
    solpos['zenith'], 
    times,
    pressure_series,
    temp_dew=temp_dew_series
)

# Calculate DHI using complete irradiance (closure equation)
df_complete = irradiance.complete_irradiance(
    solar_zenith=solpos['apparent_zenith'],
    ghi=ghi_series,
    dni=dni_dirint,
    dhi=None
)


# Checking GHI, DNI, DHI

In [17]:
import pathlib
from matplotlib import pyplot as plt
import pandas as pd
from pvlib.iotools import read_tmy3
from pvlib.solarposition import get_solarposition
from pvlib import irradiance
import pvlib

# For this example we use the Greensboro, North Carolina, TMY3 file which is
# in the pvlib data directory. TMY3 are made from the median months from years
# of data measured from 1990 to 2010. Therefore we change the timestamps to a
# common year, 1990.
DATA_DIR = pathlib.Path(pvlib.__file__).parent / 'data'
greensboro, metadata = read_tmy3(DATA_DIR / '723170TYA.CSV', coerce_year=1990,
                                 map_variables=True)

# Many of the diffuse fraction estimation methods require the "true" zenith, so
# we calculate the solar positions for the 1990 at Greensboro, NC.
# NOTE: TMY3 files timestamps indicate the end of the hour, so shift indices
# back 30-minutes to calculate solar position at center of the interval
solpos = get_solarposition(
    greensboro.index.shift(freq="-30T"), latitude=metadata['latitude'],
    longitude=metadata['longitude'], altitude=metadata['altitude'],
    pressure=greensboro.pressure*100,  # convert from millibar to Pa
    temperature=greensboro.temp_air)
solpos.index = greensboro.index  # reset index to end of the hour

/var/folders/ww/12z6q0wj4c527ftkk7sxqb680000gp/T/ipykernel_73991/3123849339.py:22: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  greensboro.index.shift(freq="-30T"), latitude=metadata['latitude'],


In [21]:
dni_dirint = irradiance.dirint(
    greensboro.ghi, solpos.zenith, greensboro.index, greensboro.pressure*100,
    temp_dew=greensboro.temp_dew)
# use "complete sum" AKA "closure" equation: DHI = GHI - DNI * cos(zenith)
df_dirint = irradiance.complete_irradiance(
    solar_zenith=solpos.apparent_zenith, ghi=greensboro.ghi, dni=dni_dirint,
    dhi=None)
out_dirint = pd.DataFrame(
    {'dni_dirint': dni_dirint, 'dhi_dirint': df_dirint.dhi},
    index=greensboro.index).fillna(0)

In [29]:
df_dirint.fillna(0).loc[::12].head(24)

,ghi,dhi,dni
1990-01-01 01:00:00-05:00,0,0.000000,0.000000
1990-01-01 13:00:00-05:00,155,155.000000,0.000000
1990-01-02 01:00:00-05:00,0,0.000000,0.000000
1990-01-02 13:00:00-05:00,175,173.937434,2.062237
1990-01-03 01:00:00-05:00,0,0.000000,0.000000
1990-01-03 13:00:00-05:00,126,126.000000,0.000000
1990-01-04 01:00:00-05:00,0,0.000000,0.000000
1990-01-04 13:00:00-05:00,333,250.476337,159.221946
1990-01-05 01:00:00-05:00,0,0.000000,0.000000
1990-01-05 13:00:00-05:00,239,222.929302,30.905951


# Where is IR

In [37]:
pd.read_feather("/Users/jmccarty/Desktop/test_weather_builder/zurich_tmy/timeseries/timeseries_1994-01-01_2024-12-31_47.40_8.51_a_l_l.feather")

,Year,Month,Day,Hour,Minute,Latitude,Longitude,Temperature,Dew Point,Pressure,Relative Humidity,Wind Speed,Wind Direction,GHI,DNI,DHI,Precipitation
0,1994,1,1,0,0,47.4,8.51,3.73788,2.65963,940.36030,92.660352,1.763703,356.032236,0.0,NaN,NaN,2.336785
1,1994,1,1,1,0,47.4,8.51,3.41714,2.37438,939.26234,92.876284,1.677461,339.187937,0.0,NaN,NaN,1.289132
2,1994,1,1,2,0,47.4,8.51,3.36862,2.19610,938.53780,92.019031,1.182062,336.698370,0.0,NaN,NaN,1.078871
3,1994,1,1,3,0,47.4,8.51,3.08584,1.95352,938.86086,92.265979,1.109665,64.507897,0.0,NaN,NaN,1.883335
4,1994,1,1,4,0,47.4,8.51,1.94820,1.87140,940.28640,99.452830,2.308115,76.834784,0.0,NaN,NaN,2.710547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271555,2024,12,31,19,0,47.4,8.51,-2.57053,-3.40262,963.11390,93.998521,0.992736,42.430919,0.0,NaN,NaN,0.000596
271556,2024,12,31,20,0,47.4,8.51,-2.32288,-3.64393,963.25430,90.641501,1.218422,30.278574,0.0,NaN,NaN,0.000000
271557,2024,12,31,21,0,47.4,8.51,-2.19730,-3.71560,962.70280,89.323625,1.289129,27.811985,0.0,NaN,NaN,0.000000
271558,2024,12,31,22,0,47.4,8.51,-1.46714,-3.29490,962.32190,87.348096,1.013619,39.067927,0.0,NaN,NaN,0.000000


# Location

In [ ]:
pvlib.location.lookup_altitude(47.4, 8.51)

502.0

In [52]:
from datetime import datetime, date
date.today().year

2025